In [1]:
import matplotlib.pyplot as plt
from PIL import Image
from PIL.Image import NEAREST
import numpy as np
import cv2
import os
import imutils
from random import random, seed, randint

from scipy import ndimage as ndi
from scipy.spatial.distance import euclidean
from skimage.morphology import watershed, disk
from skimage.feature import peak_local_max
from torchvision.transforms.functional import pad, resized_crop, resize, center_crop, rotate, hflip, to_pil_image
# from torch.random import torch.random.manual_seed(seed)

from processing import *

In [2]:
# o=get_origs()
# get_clahes(o)
# cell_masks = get_cell_masks()
# cell_markers = get_cell_markers(cell_masks, 24)
# weight_maps = get_weight_maps(cell_markers, False)

# pimg(weight_maps[1])

def plot_two_images(imgL, imgR, titleL, titleR):
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(imgL) #, cmap='gray')
    plt.title(titleL)
    f.add_subplot(1,2, 2)
    plt.imshow(imgR) #, cmap='gray')
    plt.title(titleR)
    plt.show(block=True)

In [3]:
def get_pred_masks():
    return get_binary_cell_masks(get_cell_masks())

def get_pred_markers(pred_masks, e_r):
    return get_binary_cell_markers(get_cell_masks(), e_r)

In [4]:
def threshold_binary_image(img, thresh=0.5):
    image = img.copy()
    return (image*1.0 >= thresh).astype('uint8')

In [9]:
def img_aug(dataset, new_prefix = "_aug"):
    #constants
    num_aug_multiplier = 10
    
    new_dir_name = dataset + new_prefix
    
    try:
        os.mkdir(new_dir_name)
        os.mkdir(os.path.join(new_dir_name, "originals"))
        os.mkdir(os.path.join(new_dir_name, "masks"))
    except:
        print("One of the directories to be created already exists. Clear and try again.")
        return
    
    data = []
    paths = [dataset + '/01', dataset + '/02']
    count = 0
    
    for path in paths:
        mask_path = path + '_ST'
        mask_path = os.path.join(mask_path, "SEG")
        for f in os.listdir(mask_path):
            if not f.endswith(".tif"):
                continue
            image_np = cv2.imread(os.path.join(path, f.replace('man_seg', 't')), cv2.IMREAD_GRAYSCALE)
            mask_np = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
            
#             image_pil = Image.open( os.path.join(path, f.replace('man_seg', 't')) )
#             mask_pil = Image.open( os.path.join(mask_path, f) )
            image_pil = to_pil_image(image_np)
            mask_pil = Image.open( os.path.join(mask_path, f) )
            mask_pil = mask_pil.convert("L")

            print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('man_seg', 't')))
            
            index = 't' + str(count) + ".tif"
            count += 1
            
            cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image_np)
            cv2.imwrite(os.path.join(new_dir_name, "masks", index ), mask_np)
            
            for i in range(num_aug_multiplier):
                new_img = None
                new_mask = None
                image = image_pil
                mask = mask_pil
                
                seed()
                seed_ = randint(0,1000)
                
                index = 't' + str(count) + ".tif"
                count += 1
                
                seed(seed_)
                if random() > 0.5:
                    pad_size = int(randint(0,int(image.height*0.2)))
                    pad_size = (pad_size)
                    new_img = pad(image, pad_size, padding_mode='reflect')
                    new_img = resize(new_img, (image.height, image.width))
                else:
                    t_size = image.height * (0.6 + ( random() * 0.39 ) )
                    new_img = center_crop(image, (t_size, t_size))
                    new_img = resize(new_img, (image.height, image.width) )
                
                pad_size = int(image.height*0.5)
                pad_size = (pad_size)
                new_img = pad(new_img, pad_size, padding_mode='reflect')
                new_img = rotate(new_img, randint(0,360), expand=False, fill=0)
#                 new_img = resize(new_img, (image.height, image.width) )
                new_img = center_crop(new_img, (image.height, image.width) )
                
                new_img = hflip(new_img)
                new_img.save( os.path.join(new_dir_name, "originals", index) )
                
                seed(seed_)
                if random() > 0.5:
                    pad_size = int(randint(0, int(mask.height*0.2)))
                    pad_size = (pad_size)
                    new_mask = pad(mask, pad_size, padding_mode='reflect')
                    new_mask = resize(new_mask, (mask.height, mask.width), interpolation=NEAREST)
                else:
                    t_size = mask.height * (0.6 + ( random() * 0.39 ) )
                    new_mask = center_crop(mask, (t_size, t_size) )
                    new_mask = resize(new_mask, (mask.height, mask.width), interpolation=NEAREST)
                
                pad_size = int(mask.height*0.5)
                pad_size = (pad_size)
                new_mask = pad(new_mask, pad_size, padding_mode='reflect')
                new_mask = rotate(new_mask, randint(0,360), expand=False, fill=0)
#                 new_mask = resize(new_mask, (mask.height, mask.width), interpolation=NEAREST )
                new_mask = center_crop(new_mask, (mask.height, mask.width) )
                
                new_mask = hflip(new_mask)
                new_mask.save( os.path.join(new_dir_name, "masks", index ) )
                
#                 plot_two_images(mask, new_mask, "Gen Img", "Gen Mask")
#                 plot_two_images(image, get_markers(np.asarray(new_mask)), "img", "markers")
#                 print(len(np.unique(np.asarray(mask))) - len(np.unique(np.asarray(new_mask))), np.unique(np.asarray(mask)), np.unique(np.asarray(new_mask)))
                
#                 assert(len(np.unique(np.asarray(mask))) - len(np.unique(np.asarray(new_mask))) == 0)
    
                print(index)
                
img_aug("DIC-2")
# #             np.save(os.path.join(new_dir_name, "originals", index ), image)
# #             np.save(os.path.join(new_dir_name, "clahes", index ), clahe)
# #             np.save(os.path.join(new_dir_name, "masks", index ), cell_mask)
# #             np.save(os.path.join(new_dir_name, "markers", index ), markers)
# #             np.save(os.path.join(new_dir_name, "weight_maps", index ), weight_map)
            
#             cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image)
#             cv2.imwrite(os.path.join(new_dir_name, "masks", index ), cell_mask)
#             cv2.imwrite(os.path.join(new_dir_name, "markers", index ), markers)
#             cv2.imwrite(os.path.join(new_dir_name, "weight_maps", index ), weight_map)

   Loaded DIC-2/01_ST\SEG\man_seg000.tif, DIC-2/01\t000.tif
t1.tif
t2.tif
t3.tif
t4.tif
t5.tif
t6.tif
t7.tif
t8.tif
t9.tif
t10.tif
   Loaded DIC-2/01_ST\SEG\man_seg001.tif, DIC-2/01\t001.tif
t12.tif
t13.tif
t14.tif
t15.tif
t16.tif
t17.tif
t18.tif
t19.tif
t20.tif
t21.tif
   Loaded DIC-2/01_ST\SEG\man_seg002.tif, DIC-2/01\t002.tif
t23.tif
t24.tif
t25.tif
t26.tif
t27.tif
t28.tif
t29.tif
t30.tif
t31.tif
t32.tif
   Loaded DIC-2/01_ST\SEG\man_seg003.tif, DIC-2/01\t003.tif
t34.tif
t35.tif
t36.tif
t37.tif
t38.tif
t39.tif
t40.tif
t41.tif
t42.tif
t43.tif
   Loaded DIC-2/01_ST\SEG\man_seg004.tif, DIC-2/01\t004.tif
t45.tif
t46.tif
t47.tif
t48.tif
t49.tif
t50.tif
t51.tif
t52.tif
t53.tif
t54.tif
   Loaded DIC-2/01_ST\SEG\man_seg005.tif, DIC-2/01\t005.tif
t56.tif
t57.tif
t58.tif
t59.tif
t60.tif
t61.tif
t62.tif
t63.tif
t64.tif
t65.tif
   Loaded DIC-2/01_ST\SEG\man_seg006.tif, DIC-2/01\t006.tif
t67.tif
t68.tif
t69.tif
t70.tif
t71.tif
t72.tif
t73.tif
t74.tif
t75.tif
t76.tif
   Loaded DIC-2/01_ST\SEG\man

t613.tif
t614.tif
t615.tif
   Loaded DIC-2/01_ST\SEG\man_seg056.tif, DIC-2/01\t056.tif
t617.tif
t618.tif
t619.tif
t620.tif
t621.tif
t622.tif
t623.tif
t624.tif
t625.tif
t626.tif
   Loaded DIC-2/01_ST\SEG\man_seg057.tif, DIC-2/01\t057.tif
t628.tif
t629.tif
t630.tif
t631.tif
t632.tif
t633.tif
t634.tif
t635.tif
t636.tif
t637.tif
   Loaded DIC-2/01_ST\SEG\man_seg058.tif, DIC-2/01\t058.tif
t639.tif
t640.tif
t641.tif
t642.tif
t643.tif
t644.tif
t645.tif
t646.tif
t647.tif
t648.tif
   Loaded DIC-2/01_ST\SEG\man_seg059.tif, DIC-2/01\t059.tif
t650.tif
t651.tif
t652.tif
t653.tif
t654.tif
t655.tif
t656.tif
t657.tif
t658.tif
t659.tif
   Loaded DIC-2/01_ST\SEG\man_seg060.tif, DIC-2/01\t060.tif
t661.tif
t662.tif
t663.tif
t664.tif
t665.tif
t666.tif
t667.tif
t668.tif
t669.tif
t670.tif
   Loaded DIC-2/01_ST\SEG\man_seg061.tif, DIC-2/01\t061.tif
t672.tif
t673.tif
t674.tif
t675.tif
t676.tif
t677.tif
t678.tif
t679.tif
t680.tif
t681.tif
   Loaded DIC-2/01_ST\SEG\man_seg062.tif, DIC-2/01\t062.tif
t683.tif
t684

t1200.tif
t1201.tif
t1202.tif
t1203.tif
t1204.tif
t1205.tif
t1206.tif
t1207.tif
t1208.tif
t1209.tif
   Loaded DIC-2/02_ST\SEG\man_seg026.tif, DIC-2/02\t026.tif
t1211.tif
t1212.tif
t1213.tif
t1214.tif
t1215.tif
t1216.tif
t1217.tif
t1218.tif
t1219.tif
t1220.tif
   Loaded DIC-2/02_ST\SEG\man_seg027.tif, DIC-2/02\t027.tif
t1222.tif
t1223.tif
t1224.tif
t1225.tif
t1226.tif
t1227.tif
t1228.tif
t1229.tif
t1230.tif
t1231.tif
   Loaded DIC-2/02_ST\SEG\man_seg028.tif, DIC-2/02\t028.tif
t1233.tif
t1234.tif
t1235.tif
t1236.tif
t1237.tif
t1238.tif
t1239.tif
t1240.tif
t1241.tif
t1242.tif
   Loaded DIC-2/02_ST\SEG\man_seg029.tif, DIC-2/02\t029.tif
t1244.tif
t1245.tif
t1246.tif
t1247.tif
t1248.tif
t1249.tif
t1250.tif
t1251.tif
t1252.tif
t1253.tif
   Loaded DIC-2/02_ST\SEG\man_seg030.tif, DIC-2/02\t030.tif
t1255.tif
t1256.tif
t1257.tif
t1258.tif
t1259.tif
t1260.tif
t1261.tif
t1262.tif
t1263.tif
t1264.tif
   Loaded DIC-2/02_ST\SEG\man_seg031.tif, DIC-2/02\t031.tif
t1266.tif
t1267.tif
t1268.tif
t1269.tif


t1772.tif
t1773.tif
t1774.tif
t1775.tif
t1776.tif
t1777.tif
t1778.tif
t1779.tif
t1780.tif
t1781.tif
   Loaded DIC-2/02_ST\SEG\man_seg078.tif, DIC-2/02\t078.tif
t1783.tif
t1784.tif
t1785.tif
t1786.tif
t1787.tif
t1788.tif
t1789.tif
t1790.tif
t1791.tif
t1792.tif
   Loaded DIC-2/02_ST\SEG\man_seg079.tif, DIC-2/02\t079.tif
t1794.tif
t1795.tif
t1796.tif
t1797.tif
t1798.tif
t1799.tif
t1800.tif
t1801.tif
t1802.tif
t1803.tif
   Loaded DIC-2/02_ST\SEG\man_seg080.tif, DIC-2/02\t080.tif
t1805.tif
t1806.tif
t1807.tif
t1808.tif
t1809.tif
t1810.tif
t1811.tif
t1812.tif
t1813.tif
t1814.tif
   Loaded DIC-2/02_ST\SEG\man_seg081.tif, DIC-2/02\t081.tif
t1816.tif
t1817.tif
t1818.tif
t1819.tif
t1820.tif
t1821.tif
t1822.tif
t1823.tif
t1824.tif
t1825.tif
   Loaded DIC-2/02_ST\SEG\man_seg082.tif, DIC-2/02\t082.tif
t1827.tif
t1828.tif
t1829.tif
t1830.tif
t1831.tif
t1832.tif
t1833.tif
t1834.tif
t1835.tif
t1836.tif
   Loaded DIC-2/02_ST\SEG\man_seg083.tif, DIC-2/02\t083.tif
t1838.tif
t1839.tif
t1840.tif
t1841.tif


In [14]:
def create_preprocessed_cache(dataset, new_dir_name=None):
    
    if new_dir_name == None:
        new_dir_name = dataset+"_cache"
    
    try:
        os.mkdir(new_dir_name)
        os.mkdir(os.path.join(new_dir_name, "originals"))
        os.mkdir(os.path.join(new_dir_name, "clahes"))
        os.mkdir(os.path.join(new_dir_name, "masks"))
        os.mkdir(os.path.join(new_dir_name, "markers"))
        os.mkdir(os.path.join(new_dir_name, "weight_maps"))
    except:
        print("One of the directories to be created already exists. Clear and try again.")
        return
    
    data = []
    paths = [dataset + '/01', dataset + '/02']
    count = 0
    
    for path in paths:
        mask_path = path + '_ST'
        mask_path = os.path.join(mask_path, "SEG")
        for f in os.listdir(mask_path):
            if not f.endswith(".tif"):
                continue
            image = cv2.imread(os.path.join(path, f.replace('man_seg', 't')), cv2.IMREAD_GRAYSCALE)
            clahe = equalize_clahe(image).astype(np.float32)
            mask = cv2.imread(os.path.join(mask_path, f), cv2.IMREAD_UNCHANGED)
            print("   Loaded " + os.path.join(mask_path, f) + ", " + os.path.join(path, f.replace('man_seg', 't')))
            
            # Generate the Cell Mask and Markers from the Mask
            cell_mask = (mask > 0).astype(np.uint8)
            markers = (get_markers(mask) > 0).astype(np.uint8)
            weight_map = get_weight_map(markers)
            
            index = 't' + str(count) + ".npy"
            count += 1 
            
            np.save(os.path.join(new_dir_name, "originals", index ), image)
            np.save(os.path.join(new_dir_name, "clahes", index ), clahe)
            np.save(os.path.join(new_dir_name, "masks", index ), cell_mask)
            np.save(os.path.join(new_dir_name, "markers", index ), markers)
            np.save(os.path.join(new_dir_name, "weight_maps", index ), weight_map)
            
#             cv2.imwrite(os.path.join(new_dir_name, "originals", index ), image)
#             cv2.imwrite(os.path.join(new_dir_name, "clahes", index ), clahe)
#             cv2.imwrite(os.path.join(new_dir_name, "masks", index ), cell_mask)
#             cv2.imwrite(os.path.join(new_dir_name, "markers", index ), markers)
#             cv2.imwrite(os.path.join(new_dir_name, "weight_maps", index ), weight_map)
    
            print(count)
#             plot_two_images(mask, cell_mask, "orig", "mask")
#             plot_two_images(markers, weight_map, "marker", "weight map")

create_preprocessed_cache('DIC-3', new_dir_name=None)

   Loaded DIC-3/01_ST\SEG\t0.tif, DIC-3/01\t0.tif
1
   Loaded DIC-3/01_ST\SEG\t1.tif, DIC-3/01\t1.tif
2
   Loaded DIC-3/01_ST\SEG\t10.tif, DIC-3/01\t10.tif
3
   Loaded DIC-3/01_ST\SEG\t100.tif, DIC-3/01\t100.tif
4
   Loaded DIC-3/01_ST\SEG\t1000.tif, DIC-3/01\t1000.tif
5
   Loaded DIC-3/01_ST\SEG\t1001.tif, DIC-3/01\t1001.tif
6
   Loaded DIC-3/01_ST\SEG\t1002.tif, DIC-3/01\t1002.tif
7
   Loaded DIC-3/01_ST\SEG\t1003.tif, DIC-3/01\t1003.tif
8
   Loaded DIC-3/01_ST\SEG\t1004.tif, DIC-3/01\t1004.tif
9
   Loaded DIC-3/01_ST\SEG\t1005.tif, DIC-3/01\t1005.tif
10
   Loaded DIC-3/01_ST\SEG\t1006.tif, DIC-3/01\t1006.tif
11
   Loaded DIC-3/01_ST\SEG\t1007.tif, DIC-3/01\t1007.tif
12
   Loaded DIC-3/01_ST\SEG\t1008.tif, DIC-3/01\t1008.tif
13
   Loaded DIC-3/01_ST\SEG\t1009.tif, DIC-3/01\t1009.tif
14
   Loaded DIC-3/01_ST\SEG\t101.tif, DIC-3/01\t101.tif
15
   Loaded DIC-3/01_ST\SEG\t1010.tif, DIC-3/01\t1010.tif
16
   Loaded DIC-3/01_ST\SEG\t1011.tif, DIC-3/01\t1011.tif
17
   Loaded DIC-3/01_ST\SEG\

140
   Loaded DIC-3/01_ST\SEG\t1123.tif, DIC-3/01\t1123.tif
141
   Loaded DIC-3/01_ST\SEG\t1124.tif, DIC-3/01\t1124.tif
142
   Loaded DIC-3/01_ST\SEG\t1125.tif, DIC-3/01\t1125.tif
143
   Loaded DIC-3/01_ST\SEG\t1126.tif, DIC-3/01\t1126.tif
144
   Loaded DIC-3/01_ST\SEG\t1127.tif, DIC-3/01\t1127.tif
145
   Loaded DIC-3/01_ST\SEG\t1128.tif, DIC-3/01\t1128.tif
146
   Loaded DIC-3/01_ST\SEG\t1129.tif, DIC-3/01\t1129.tif
147
   Loaded DIC-3/01_ST\SEG\t113.tif, DIC-3/01\t113.tif
148
   Loaded DIC-3/01_ST\SEG\t1130.tif, DIC-3/01\t1130.tif
149
   Loaded DIC-3/01_ST\SEG\t1131.tif, DIC-3/01\t1131.tif
150
   Loaded DIC-3/01_ST\SEG\t1132.tif, DIC-3/01\t1132.tif
151
   Loaded DIC-3/01_ST\SEG\t1133.tif, DIC-3/01\t1133.tif
152
   Loaded DIC-3/01_ST\SEG\t1134.tif, DIC-3/01\t1134.tif
153
   Loaded DIC-3/01_ST\SEG\t1135.tif, DIC-3/01\t1135.tif
154
   Loaded DIC-3/01_ST\SEG\t1136.tif, DIC-3/01\t1136.tif
155
   Loaded DIC-3/01_ST\SEG\t1137.tif, DIC-3/01\t1137.tif
156
   Loaded DIC-3/01_ST\SEG\t1138.tif, D

278
   Loaded DIC-3/01_ST\SEG\t1248.tif, DIC-3/01\t1248.tif
279
   Loaded DIC-3/01_ST\SEG\t1249.tif, DIC-3/01\t1249.tif
280
   Loaded DIC-3/01_ST\SEG\t125.tif, DIC-3/01\t125.tif
281
   Loaded DIC-3/01_ST\SEG\t1250.tif, DIC-3/01\t1250.tif
282
   Loaded DIC-3/01_ST\SEG\t1251.tif, DIC-3/01\t1251.tif
283
   Loaded DIC-3/01_ST\SEG\t1252.tif, DIC-3/01\t1252.tif
284
   Loaded DIC-3/01_ST\SEG\t1253.tif, DIC-3/01\t1253.tif
285
   Loaded DIC-3/01_ST\SEG\t1254.tif, DIC-3/01\t1254.tif
286
   Loaded DIC-3/01_ST\SEG\t1255.tif, DIC-3/01\t1255.tif
287
   Loaded DIC-3/01_ST\SEG\t1256.tif, DIC-3/01\t1256.tif
288
   Loaded DIC-3/01_ST\SEG\t1257.tif, DIC-3/01\t1257.tif
289
   Loaded DIC-3/01_ST\SEG\t1258.tif, DIC-3/01\t1258.tif
290
   Loaded DIC-3/01_ST\SEG\t1259.tif, DIC-3/01\t1259.tif
291
   Loaded DIC-3/01_ST\SEG\t126.tif, DIC-3/01\t126.tif
292
   Loaded DIC-3/01_ST\SEG\t1260.tif, DIC-3/01\t1260.tif
293
   Loaded DIC-3/01_ST\SEG\t1261.tif, DIC-3/01\t1261.tif
294
   Loaded DIC-3/01_ST\SEG\t1262.tif, DIC

416
   Loaded DIC-3/01_ST\SEG\t1372.tif, DIC-3/01\t1372.tif
417
   Loaded DIC-3/01_ST\SEG\t1373.tif, DIC-3/01\t1373.tif
418
   Loaded DIC-3/01_ST\SEG\t1374.tif, DIC-3/01\t1374.tif
419
   Loaded DIC-3/01_ST\SEG\t1375.tif, DIC-3/01\t1375.tif
420
   Loaded DIC-3/01_ST\SEG\t1376.tif, DIC-3/01\t1376.tif
421
   Loaded DIC-3/01_ST\SEG\t1377.tif, DIC-3/01\t1377.tif
422
   Loaded DIC-3/01_ST\SEG\t1378.tif, DIC-3/01\t1378.tif
423
   Loaded DIC-3/01_ST\SEG\t1379.tif, DIC-3/01\t1379.tif
424
   Loaded DIC-3/01_ST\SEG\t138.tif, DIC-3/01\t138.tif
425
   Loaded DIC-3/01_ST\SEG\t1380.tif, DIC-3/01\t1380.tif
426
   Loaded DIC-3/01_ST\SEG\t1381.tif, DIC-3/01\t1381.tif
427
   Loaded DIC-3/01_ST\SEG\t1382.tif, DIC-3/01\t1382.tif
428
   Loaded DIC-3/01_ST\SEG\t1383.tif, DIC-3/01\t1383.tif
429
   Loaded DIC-3/01_ST\SEG\t1384.tif, DIC-3/01\t1384.tif
430
   Loaded DIC-3/01_ST\SEG\t1385.tif, DIC-3/01\t1385.tif
431
   Loaded DIC-3/01_ST\SEG\t1386.tif, DIC-3/01\t1386.tif
432
   Loaded DIC-3/01_ST\SEG\t1387.tif, D

554
   Loaded DIC-3/01_ST\SEG\t1497.tif, DIC-3/01\t1497.tif
555
   Loaded DIC-3/01_ST\SEG\t1498.tif, DIC-3/01\t1498.tif
556
   Loaded DIC-3/01_ST\SEG\t1499.tif, DIC-3/01\t1499.tif
557
   Loaded DIC-3/01_ST\SEG\t15.tif, DIC-3/01\t15.tif
558
   Loaded DIC-3/01_ST\SEG\t150.tif, DIC-3/01\t150.tif
559
   Loaded DIC-3/01_ST\SEG\t1500.tif, DIC-3/01\t1500.tif
560
   Loaded DIC-3/01_ST\SEG\t1501.tif, DIC-3/01\t1501.tif
561
   Loaded DIC-3/01_ST\SEG\t1502.tif, DIC-3/01\t1502.tif
562
   Loaded DIC-3/01_ST\SEG\t1503.tif, DIC-3/01\t1503.tif
563
   Loaded DIC-3/01_ST\SEG\t1504.tif, DIC-3/01\t1504.tif
564
   Loaded DIC-3/01_ST\SEG\t1505.tif, DIC-3/01\t1505.tif
565
   Loaded DIC-3/01_ST\SEG\t1506.tif, DIC-3/01\t1506.tif
566
   Loaded DIC-3/01_ST\SEG\t1507.tif, DIC-3/01\t1507.tif
567
   Loaded DIC-3/01_ST\SEG\t1508.tif, DIC-3/01\t1508.tif
568
   Loaded DIC-3/01_ST\SEG\t1509.tif, DIC-3/01\t1509.tif
569
   Loaded DIC-3/01_ST\SEG\t151.tif, DIC-3/01\t151.tif
570
   Loaded DIC-3/01_ST\SEG\t1510.tif, DIC-3/0

...

1248
   Loaded DIC-3/01_ST\SEG\t459.tif, DIC-3/01\t459.tif
1249
   Loaded DIC-3/01_ST\SEG\t46.tif, DIC-3/01\t46.tif
1250
   Loaded DIC-3/01_ST\SEG\t460.tif, DIC-3/01\t460.tif
1251
   Loaded DIC-3/01_ST\SEG\t461.tif, DIC-3/01\t461.tif
1252
   Loaded DIC-3/01_ST\SEG\t462.tif, DIC-3/01\t462.tif
1253
   Loaded DIC-3/01_ST\SEG\t463.tif, DIC-3/01\t463.tif
1254
   Loaded DIC-3/01_ST\SEG\t464.tif, DIC-3/01\t464.tif
1255
   Loaded DIC-3/01_ST\SEG\t465.tif, DIC-3/01\t465.tif
1256
   Loaded DIC-3/01_ST\SEG\t466.tif, DIC-3/01\t466.tif
1257
   Loaded DIC-3/01_ST\SEG\t467.tif, DIC-3/01\t467.tif
1258
   Loaded DIC-3/01_ST\SEG\t468.tif, DIC-3/01\t468.tif
1259
   Loaded DIC-3/01_ST\SEG\t469.tif, DIC-3/01\t469.tif
1260
   Loaded DIC-3/01_ST\SEG\t47.tif, DIC-3/01\t47.tif
1261
   Loaded DIC-3/01_ST\SEG\t470.tif, DIC-3/01\t470.tif
1262
   Loaded DIC-3/01_ST\SEG\t471.tif, DIC-3/01\t471.tif
1263
   Loaded DIC-3/01_ST\SEG\t472.tif, DIC-3/01\t472.tif
1264
   Loaded DIC-3/01_ST\SEG\t473.tif, DIC-3/01\t473.tif
1

1388
   Loaded DIC-3/01_ST\SEG\t585.tif, DIC-3/01\t585.tif
1389
   Loaded DIC-3/01_ST\SEG\t586.tif, DIC-3/01\t586.tif
1390
   Loaded DIC-3/01_ST\SEG\t587.tif, DIC-3/01\t587.tif
1391
   Loaded DIC-3/01_ST\SEG\t588.tif, DIC-3/01\t588.tif
1392
   Loaded DIC-3/01_ST\SEG\t589.tif, DIC-3/01\t589.tif
1393
   Loaded DIC-3/01_ST\SEG\t59.tif, DIC-3/01\t59.tif
1394
   Loaded DIC-3/01_ST\SEG\t590.tif, DIC-3/01\t590.tif
1395
   Loaded DIC-3/01_ST\SEG\t591.tif, DIC-3/01\t591.tif
1396
   Loaded DIC-3/01_ST\SEG\t592.tif, DIC-3/01\t592.tif
1397
   Loaded DIC-3/01_ST\SEG\t593.tif, DIC-3/01\t593.tif
1398
   Loaded DIC-3/01_ST\SEG\t594.tif, DIC-3/01\t594.tif
1399
   Loaded DIC-3/01_ST\SEG\t595.tif, DIC-3/01\t595.tif
1400
   Loaded DIC-3/01_ST\SEG\t596.tif, DIC-3/01\t596.tif
1401
   Loaded DIC-3/01_ST\SEG\t597.tif, DIC-3/01\t597.tif
1402
   Loaded DIC-3/01_ST\SEG\t598.tif, DIC-3/01\t598.tif
1403
   Loaded DIC-3/01_ST\SEG\t599.tif, DIC-3/01\t599.tif
1404
   Loaded DIC-3/01_ST\SEG\t6.tif, DIC-3/01\t6.tif
140

1528
   Loaded DIC-3/01_ST\SEG\t710.tif, DIC-3/01\t710.tif
1529
   Loaded DIC-3/01_ST\SEG\t711.tif, DIC-3/01\t711.tif
1530
   Loaded DIC-3/01_ST\SEG\t712.tif, DIC-3/01\t712.tif
1531
   Loaded DIC-3/01_ST\SEG\t713.tif, DIC-3/01\t713.tif
1532
   Loaded DIC-3/01_ST\SEG\t714.tif, DIC-3/01\t714.tif
1533
   Loaded DIC-3/01_ST\SEG\t715.tif, DIC-3/01\t715.tif
1534
   Loaded DIC-3/01_ST\SEG\t716.tif, DIC-3/01\t716.tif
1535
   Loaded DIC-3/01_ST\SEG\t717.tif, DIC-3/01\t717.tif
1536
   Loaded DIC-3/01_ST\SEG\t718.tif, DIC-3/01\t718.tif
1537
   Loaded DIC-3/01_ST\SEG\t719.tif, DIC-3/01\t719.tif
1538
   Loaded DIC-3/01_ST\SEG\t72.tif, DIC-3/01\t72.tif
1539
   Loaded DIC-3/01_ST\SEG\t720.tif, DIC-3/01\t720.tif
1540
   Loaded DIC-3/01_ST\SEG\t721.tif, DIC-3/01\t721.tif
1541
   Loaded DIC-3/01_ST\SEG\t722.tif, DIC-3/01\t722.tif
1542
   Loaded DIC-3/01_ST\SEG\t723.tif, DIC-3/01\t723.tif
1543
   Loaded DIC-3/01_ST\SEG\t724.tif, DIC-3/01\t724.tif
1544
   Loaded DIC-3/01_ST\SEG\t725.tif, DIC-3/01\t725.tif

1668
   Loaded DIC-3/01_ST\SEG\t837.tif, DIC-3/01\t837.tif
1669
   Loaded DIC-3/01_ST\SEG\t838.tif, DIC-3/01\t838.tif
1670
   Loaded DIC-3/01_ST\SEG\t839.tif, DIC-3/01\t839.tif
1671
   Loaded DIC-3/01_ST\SEG\t84.tif, DIC-3/01\t84.tif
1672
   Loaded DIC-3/01_ST\SEG\t840.tif, DIC-3/01\t840.tif
1673
   Loaded DIC-3/01_ST\SEG\t841.tif, DIC-3/01\t841.tif
1674
   Loaded DIC-3/01_ST\SEG\t842.tif, DIC-3/01\t842.tif
1675
   Loaded DIC-3/01_ST\SEG\t843.tif, DIC-3/01\t843.tif
1676
   Loaded DIC-3/01_ST\SEG\t844.tif, DIC-3/01\t844.tif
1677
   Loaded DIC-3/01_ST\SEG\t845.tif, DIC-3/01\t845.tif
1678
   Loaded DIC-3/01_ST\SEG\t846.tif, DIC-3/01\t846.tif
1679
   Loaded DIC-3/01_ST\SEG\t847.tif, DIC-3/01\t847.tif
1680
   Loaded DIC-3/01_ST\SEG\t848.tif, DIC-3/01\t848.tif
1681
   Loaded DIC-3/01_ST\SEG\t849.tif, DIC-3/01\t849.tif
1682
   Loaded DIC-3/01_ST\SEG\t85.tif, DIC-3/01\t85.tif
1683
   Loaded DIC-3/01_ST\SEG\t850.tif, DIC-3/01\t850.tif
1684
   Loaded DIC-3/01_ST\SEG\t851.tif, DIC-3/01\t851.tif
1

1808
   Loaded DIC-3/01_ST\SEG\t963.tif, DIC-3/01\t963.tif
1809
   Loaded DIC-3/01_ST\SEG\t964.tif, DIC-3/01\t964.tif
1810
   Loaded DIC-3/01_ST\SEG\t965.tif, DIC-3/01\t965.tif
1811
   Loaded DIC-3/01_ST\SEG\t966.tif, DIC-3/01\t966.tif
1812
   Loaded DIC-3/01_ST\SEG\t967.tif, DIC-3/01\t967.tif
1813
   Loaded DIC-3/01_ST\SEG\t968.tif, DIC-3/01\t968.tif
1814
   Loaded DIC-3/01_ST\SEG\t969.tif, DIC-3/01\t969.tif
1815
   Loaded DIC-3/01_ST\SEG\t97.tif, DIC-3/01\t97.tif
1816
   Loaded DIC-3/01_ST\SEG\t970.tif, DIC-3/01\t970.tif
1817
   Loaded DIC-3/01_ST\SEG\t971.tif, DIC-3/01\t971.tif
1818
   Loaded DIC-3/01_ST\SEG\t972.tif, DIC-3/01\t972.tif
1819
   Loaded DIC-3/01_ST\SEG\t973.tif, DIC-3/01\t973.tif
1820
   Loaded DIC-3/01_ST\SEG\t974.tif, DIC-3/01\t974.tif
1821
   Loaded DIC-3/01_ST\SEG\t975.tif, DIC-3/01\t975.tif
1822
   Loaded DIC-3/01_ST\SEG\t976.tif, DIC-3/01\t976.tif
1823
   Loaded DIC-3/01_ST\SEG\t977.tif, DIC-3/01\t977.tif
1824
   Loaded DIC-3/01_ST\SEG\t978.tif, DIC-3/01\t978.tif

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 24)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[1]
# b = get_clahes(origs)[1]
# c = cell_masks[1]
# d = cell_markers[1]

# h = weight_maps[1]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 8)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[0]
# b = get_clahes(origs)[0]
# c = cell_masks[0]
# d = cell_markers[0]

# h = weight_maps[0]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)

In [ ]:
# cell_masks = get_cell_masks()
# origs = get_origs()
# cell_markers = get_cell_markers(cell_masks, 2)
# weight_maps = get_weight_maps(cell_markers, False)

# a = origs[2]
# b = get_clahes(origs)[2]
# c = cell_masks[2]
# d = cell_markers[2]

# h = weight_maps[2]


# pimg(a)

# pimg(b)

# pimg(c)

# pimg(d)

# pimg(h)